In [1]:
# !pip install --upgrade pip

In [2]:
# !pip install python-pptx
# !pip install transformers
# !pip3 install torch torchvision torchaudio
# !conda install -c pytorch pytorch
# !conda install -c conda-forge tensorflow keras

In [1]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE

In [2]:
from collections import defaultdict

In [3]:
import os

dir_path = "/Users/ktg/Desktop/DeepLearning/Simila/data/original"

for (root, directories, files) in os.walk(dir_path):
    for file in files:
        # if '.txt' in file:
        file_path = os.path.join(root, file)
        print(file_path)

/Users/ktg/Desktop/DeepLearning/Simila/data/original/deep_learning_intro.pptx


In [4]:
import re

class Text_Preprocessing:
    def __init__(self) -> None:
        self.punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
        self.mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", 
                                "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", 
                                "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', 
                                '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', 
                                '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 
        self.specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}

    def preprocessing(self, text):
        text = self.clean(text)
        text = self.clean_str(text)
        text = self.token(text)
        return text
    
    def clean(self, text):
        for p in self.mapping:
            text = text.replace(p, self.mapping[p])
        
        for p in self.punct:
            text = text.replace(p, f' {p} ')
        
        
        for s in self.specials:
            text = text.replace(s, self.specials[s])
        return text.strip()

    def clean_str(self, text):
        # pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
        # text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '<[^>]*>'         # HTML 태그 제거
        text = re.sub(pattern=pattern, repl='', string=text)
        pattern = '[^\w\s\n]'         # 특수기호제거
        text = re.sub(pattern=pattern, repl='', string=text)
        text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ', string=text)
        text = re.sub('\n', ' ', string=text)
        return text 
    
    def token(self, text) -> list:
        ls = [t.strip() for t in text.split(' ') if t]
        res = ' '.join(ls)
        return res
        

In [8]:
class PPT_Info_Extract:
    def __init__(self, path) -> None:
        path = "data/original/deep_learning_intro.pptx"
        self.parsed = Presentation(path)
        self.text_dict = {'total':[], 'page':defaultdict(list)}
        self.link_list = []
        self.img_dict = {}
        self.text_info = None
        self.page_num = None
        self.img_num = 0
        self.tp = Text_Preprocessing()
        self.image_blob = []
        
    def setting(self):
        pass
    
    def last(self):
        for text_ls in self.text_dict['page'].values():
            for text in text_ls:
                # print(text)
                pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
                links = re.findall(pattern, text)
                if links:
                    for link in links:
                        self.link_list.append(link)
                    text = re.sub(pattern=pattern, repl='', string=text)
                
                if text:
                    self.text_dict['total'].append(text)
        
        self.text_info = self.tp.preprocessing(' '.join(self.text_dict['total']))
    
    def extract(self):
        for idx, slide in enumerate(self.parsed.slides):
            self.page_num = idx+1
            for shape in slide.shapes:
                self.group_check(shape)
        
        self.last()
    
    def group_check(self, shape):
        # 그룹 모형
        if shape.shape_type == MSO_SHAPE_TYPE.GROUP:
            for s in shape.shapes:
                self.info_extract(s)
        # 그룹 모형 아님
        else:
            self.info_extract(shape)
    
    def info_extract(self, shape):
        if shape.shape_type == MSO_SHAPE_TYPE.GROUP:
            self.group_check(shape)
        else:
            # yes image
            if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                image_blob = shape.image.blob
                if image_blob not in self.image_blob:
                    ext = shape.image.ext # - (확장명)
                    size = shape.image.size
                    if ext in ['png', 'jpeg', 'jpg'] and size[0] >= 200 and size[1] >= 200:
                        self.img_num += 1
                        num = str(self.img_num).zfill(3)
                        save_path =f"res/image_{num}.{ext}"
                        with open(save_path, "wb") as file:
                            file.write(image_blob)
                        self.image_blob.append(image_blob)
                        self.img_dict[self.img_num] = save_path
            # no image
            else:
                # yes text
                if shape.has_text_frame:
                    if shape.text.strip() != "":
                        self.text_dict['page'][self.page_num].append(shape.text.strip())

In [9]:
ppt = PPT_Info_Extract('s')

In [10]:
ppt.extract()

In [13]:
# https://huggingface.co/nlpconnect/vit-gpt2-image-captioning

In [11]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image
from tqdm import tqdm

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image_paths):
  images = []
  for image_path in tqdm(image_paths):
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

/opt/anaconda3/envs/ppt_3_7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/ppt_3_7/lib/python3.7/site-packages/transformers/models/vit/feature_extraction_vit.py:31: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  FutureWarning,


In [12]:
predict_step(list(ppt.img_dict.values()))

100%|██████████| 38/38 [00:00<00:00, 136.76it/s]


['a blue and white photo of a person holding a remote control',
 'a close up picture of a clock on a wall',
 'an aerial view of a piece of artwork on a piece of paper',
 'a painting of a person holding a toy elephant',
 'a series of photographs showing a clock on a wall',
 'an aerial view of a clock on a wall',
 'a collage of photos showing different types of electronic devices',
 'a series of images showing a person holding a blue and white disk',
 'a collage of photos showing different types of electronic devices',
 'a collage of photos of a person holding an electronic device',
 'a black and white photo of a clock on a wall',
 'a small kitten sitting on the floor in front of a mirror',
 'a white dog is looking at the camera',
 'a gray and white cat sitting on top of a sandy surface',
 'a brown and white dog laying on the floor',
 'a cat sitting on the floor looking at the camera',
 'a wooden table topped with lots of different colored balls',
 'a blue and white photo of a person on 

In [3]:
# pip install Pillow
# pip install pdfminer.six
# pip install PyMuPDF

In [19]:
import fitz

# Open the PDF file
pdf_document = fitz.open("test.pdf")

# Iterate through each page of the PDF
for page_number in range(pdf_document.page_count):
    page = pdf_document[page_number]
    
    # Get the page's size
    page_width = page.bound().width
    page_height = page.bound().height
    
    # Create a pixmap with the same size as the page
    pixmap = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)
    
    # Save the pixmap as a PNG file
    png_filename = f"page_{page_number + 1}.png"
    pixmap.save(png_filename)
    
    # Free up memory
    pixmap = None

# Close the PDF file
pdf_document.close()


In [7]:
import fitz
import io
from PIL import Image

file_path = "test3.pdf"
open_file = fitz.open(file_path)

for page_number in range(len(open_file)):
    page = open_file[page_number]
    list_image = page.get_images()

    if list_image:
        print(f"{len(list_image)} images found on page {page_number}")
        
        for i, image in enumerate(list_image):
            # Get the pixmap of the image
            pixmap = fitz.Pixmap(open_file, image[0])
            
            # Save each image as a separate file
            image_filename = f"page_{page_number + 1}_image_{i + 1}.png"
            pixmap.save(image_filename)
    else:
        print("No images found on page", page_number)

open_file.close()

2 images found on page 0
No images found on page 1
2 images found on page 2
2 images found on page 3
2 images found on page 4
No images found on page 5
1 images found on page 6
No images found on page 7
5 images found on page 8
2 images found on page 9


In [6]:
from pdfminer.high_level import extract_text

text = extract_text("test3.pdf")
print(text)

www.dbpia.co.kr

정지수외6인: 문서유사도를통한관련문서분류시스템연구   77(Jisoo Jeong et al.: Related Documents Classification System by Similarity between Documents)Copyright Ⓒ2016 Korean Institute of Broadcast and Media Engineers. All rights reserved.“This is an Open-Access article distributed under the terms of the Creative Commons BY-NC-ND (http://creativecommons.org/licenses/by-nc-nd/3.0) which permits unrestricted non-commercial use, distribution, and reproduction in any medium, provided the original work is properly cited and not altered.”a)세종대학교소프트웨어융합학과(Department of Software Convergence, Sejong University)b)세종대학교디지털콘텐츠학과(Department of Digital Contents, Sejong University)c)세종대학교인공지능언어공학과(Department of Artificial  Intelligence and Linguistic Engineering, Sejong University) d)세종대학교소프트웨어학과(Department of Software, Sejong University)‡Corresponding Author :김원일(Wonil Kim)E-mail: wikim@sejong.ac.krTel: +82-3408-3795ORCID: https://orcid.org/0000-0002-1489-8427※This research was supported by Basic Science Rese

In [8]:
import PyPDF2

pdf = open('test.pdf', 'rb')
reader = PyPDF2.PdfReader(pdf)
page = reader.pages[2]
print(page.extract_text())

1.
서
론
분
석
배
경
및
목
적
서
울
시
안
심
홈
세
트
지
원
서
비
스
정
책
분
석
2019
년부터
전국
지자체에서
여성
1
인가구를
대상으로
한
안심지원사업을
시범적으로
운영하기
시작했고
,
2021
년
6
월부터
서울시에서는
사업을
더
많은
자치구로
확대했다
.
정책에
대한
만족도는
높았으나
,
여성
1
인가구
수에
비해
지원은
턱없이
부족했다
.
이는
시범운영이
이루어지던
시기였기에
차차
지원범위를
확대해나가야
한다
.
따라서
본
연구자들은
서울시
안심홈세트
지원정책을
활성화하기
위해
R,
Python
등을
이용해
서울시의
행정구별
여러
변수를
분석하고
,
시각화하여
지자체에서
‘
우선지원분배
행정구
선택방법
’
을
제안하고자
한다
.
여
성
1
인
가
구
에
대
한
정
책
지
원
미
비
1
인가구
밀집지역은
비밀집지역에
비해
2
~
3
배
높은
범죄발생율
을
보인다
.
또한
주요
5
대
강력범죄가
주거지에서
가장
많이
발생하며
,
여성
피해자의
수도
꾸준히
증가세를
보이고
있다
.
빅카인즈를
통해
2019
년
5
월
‘
신림동
강간미수
사건
’
전후로
’
1
인가구
and
여성
’
키워드로
연관어
분석을
해보았을
때
,
주거공간에서의
범죄불안
및
지원정책에
대한
수요가
증가했음을
알
수
있었다
.
따라서
본
연구에서는
정책지원
필요성이
크게
증가했음을
인지하고
,
이에
대해
구체적인
지원이
이루어져야
하기에
여성
1
인가구에
대한
정책을
찾아
분석했다
.
1
인
가
구
의
증
가
로
인
한
연
구
필
요
성
대
두
서울시의
1
인가구
수는
2000
년
50
만명에서부터
2015
년
100
만명을
넘기고
,
2019
년에는
약
130
만명에
이를만큼
꾸준히
증가하고
있다
.
또한
주된
가구
구성방식이
핵가족
단위에서
1
인가구
단위로
바뀌어가고
있다
.
학계에서도
많은
연구가
이루어지고
있기에
지자체에서도
이에
주목하고
,
구체적
정책을
제시해야
한다
.
서울특별시
1
인가구
대책정책
연구
(
서울시
,
2014
)
에서는
1
인가구
정책을